In [1]:
import urllib.request    # needed for download of the example database
import shutil            # needed for unziping of the example database

from sqlalchemy import * # package providing unified access to various databases
from sqlalchemy.orm import *
import os                # for checking existence/removing of a file
import pandas as pd

In [2]:
engine = create_engine("sqlite:///create.db",echo=False)
session = Session(engine)

%load_ext sql
%sql sqlite:///create.db

In [3]:
def teacherSumDF():
    Teacher_summary = pd.read_sql("""SELECT Students.Universityid, Students.Name, Students.Email, Task.Taskid, Task.Title,
    Submission.Submissionid, Submission.Evaluation_finished AS Evalution, Submission.Evaluation_request, Submission.Evaluation, Answer.text
        FROM Students
        INNER JOIN Assignment USING (Universityid)
        INNER JOIN Task USING (Taskid)
        INNER JOIN Submission USING (Assignmentid)
        INNER JOIN Answer USING (Submissionid)""", engine)
    Teacher_summary.loc[Teacher_summary["Evalution"] != 0, "Evalution"] = "Finished"
    Teacher_summary.loc[Teacher_summary["Evalution"] == 0, "Evalution"] = "Pending"
    Teacher_summary.loc[Teacher_summary["Evaluation_request"] != 0, "Evaluation_request"] = "Requested"
    Teacher_summary.loc[Teacher_summary["Evaluation_request"] == 0, "Evaluation_request"] = "Not Requested"
    
    return(Teacher_summary)

teacherSumDF()

,Universityid,Name,Email,Taskid,Title,Submissionid,Evalution,Evaluation_request,Evaluation,Text
0,s3015725,Orla Wang,s3015725@leidenuniv.nl,0,Task Title 67,1,Pending,Not Requested,None,Example Answer text 67: My solution is ......
1,s3015725,Orla Wang,s3015725@leidenuniv.nl,0,Task Title 67,1,Pending,Not Requested,None,Example Answer text 67: My solution is ......
2,s3015725,Orla Wang,s3015725@leidenuniv.nl,0,Task Title 67,1,Pending,Not Requested,None,Example Answer text 67: My solution is ......
3,s3015725,Orla Wang,s3015725@leidenuniv.nl,0,Task Title 67,1,Pending,Not Requested,None,Example Answer text 67: My solution is ......
4,s3015725,Orla Wang,s3015725@leidenuniv.nl,0,Task Title 67,1,Pending,Not Requested,None,Example Answer text 67: My solution is ......
...,...,...,...,...,...,...,...,...,...,...
625,s3093799,Lena Cummings,s3093799@leidenuniv.nl,89,Task Title 90,90,Pending,Not Requested,None,Example Answer text 90: My solution is ......
626,s3093799,Lena Cummings,s3093799@leidenuniv.nl,89,Task Title 90,90,Pending,Not Requested,None,Example Answer text 90: My solution is ......
627,s3093799,Lena Cummings,s3093799@leidenuniv.nl,89,Task Title 90,90,Pending,Not Requested,None,Example Answer text 90: My solution is ......
628,s3093799,Lena Cummings,s3093799@leidenuniv.nl,89,Task Title 90,90,Pending,Not Requested,None,Example Answer text 90: My solution is ......
